In [4]:
import numpy as np
import operator
import matplotlib.pyplot as plt
from transformers import pipeline
from pylab import rcParams
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import ZeroShotClassificationExplainer
zero_shot_classifier = pipeline("zero-shot-classification",model='facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained("Recognai/zeroshot_selectra_medium")#("facebook/bart-base-mnli")#("Recognai/zeroshot_selectra_medium")
model = AutoModelForSequenceClassification.from_pretrained("Recognai/zeroshot_selectra_medium")#("facebook/bart-base-mnli")#("Recognai/zeroshot_selectra_medium")
zero_shot_explainer = ZeroShotClassificationExplainer(model, tokenizer)
import sys
#sys.exit()
import nltk
nltk.download('wordnet')
import nltk
from stop_words import get_stop_words
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(get_stop_words('en'))         #Have around 900 stopwords
nltk_words = list(stopwords.words('english'))   #Have around 150 stopwords
stop_words.extend(nltk_words)
#Sentiment explanation
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_new = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(model_new,tokenizer)

# With both the model and tokenizer initialized we are now able to get explanations on an example text.

from transformers_interpret import SequenceClassificationExplainer
def review_explain(text):
        result = zero_shot_classifier(
          sequences =text,candidate_labels = ['Positive', 'Negative'],multi_label= False)
        
        #zero_shot_explainer = ZeroShotClassificationExplainer(model, tokenizer)

       # print(result)
        #word_attributions = zero_shot_explainer(text,labels = ["Positive", "Negative"],)
        #return word_attributions,zero_shot_explainer.predicted_label
        return result['labels']#,word_attributions,zero_shot_explainer.predicted_label



ht={}
mv={}
def hotel_data():
      import re
      fh=open("hotel.txt")
      import nltk
      nltk.download('punkt')
      for k in fh:
        hh=k.strip("\n").split("::")
        #print(hh)
        jj=re.sub('\s+(a|an|and|the)(\s+)', '\2',hh[1])
        from nltk.tokenize import word_tokenize
        tokens = word_tokenize(jj)
        # convert to lower case
        tokens = [w.lower() for w in tokens]
        # remove punctuation from each word
        import string
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        # filter out stop words
        from nltk.corpus import stopwords
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        ht[hh[0]]=words#jj.split()
        #print("\n")
      fh.close()
      return ht
      
def movie_data():
        import re
        fh1=open("imdb.txt")
        import nltk
        nltk.download('punkt')
        for k in fh1:
          hh=k.strip("\n").split("::")
          #print(hh)
          jj=re.sub('\s+(a|an|and|the)(\s+)', '\2',hh[1])
          from nltk.tokenize import word_tokenize
          tokens = word_tokenize(jj)
          # convert to lower case
          tokens = [w.lower() for w in tokens]
          # remove punctuation from each word
          import string
          table = str.maketrans('', '', string.punctuation)
          stripped = [w.translate(table) for w in tokens]
          # remove remaining tokens that are not alphabetic
          words = [word for word in stripped if word.isalpha()]
          # filter out stop words
          from nltk.corpus import stopwords
          stop_words = set(stopwords.words('english'))
          words = [w for w in words if not w in stop_words]
          mv[hh[0]]=words#jj.split()
          #print("\n")
        fh1.close()
        return mv

mv=movie_data()
ht=hotel_data()




htt_cls={}
htt_rank={}         
cn=0

print("Hotel Review")
for jz in ht:
  if cn<100:#0000:
            #print("heloooooooooooooo")
            #print(ht[j])
            s=""
            zz=0
            htt_exp={}
            gg=[]
            for vv in ht[jz]:
              if len(vv)>=5:
                #if zz<100:
                if vv not in stop_words:
                    s=s+str(vv)+" "
                    zz=zz+1
            #print(j,str(s))
            #sys.exit()
            clp=review_explain(s)
            htt_cls[jz]=clp[0]
            word_attributions = zero_shot_explainer(s,labels = [clp[0]],)

            #attp,expp=review_explain(s)
            #("Today apple released the new Macbook showing off a range of new features found in the proprietary silicon chip computer.")
            #if str(clp[0])==str(expp):
            #print(clp[0]) sorted(dc.items(), key=operator.itemgetter(1),reverse=True)
            #att_words= cls_explainer(s, class_name=clp[0].upper())
            #print("Predicted Sentiment Class: "+"\n"+str(clp[0])+"\n")
            from nltk.stem import WordNetLemmatizer
            lemmatizer = WordNetLemmatizer()
            from nltk.stem import PorterStemmer
            from nltk.tokenize import word_tokenize
            
            hhh=[]
            for hh in word_attributions[clp[0]]:#att_words:
              if hh[1]>0:
                if hh[0]. isalnum():
                  vvv=lemmatizer.lemmatize(str(hh[0]))
                  for bn in ht[jz]:
                    if bn.__contains__(str(vvv)):
                       if bn not in htt_exp:
                            htt_exp[bn]=float(hh[1])
            #print(htt_exp)
            #cn=cn+1
            #sys.exit()
            rankk=sorted(htt_exp.items(), key=operator.itemgetter(1),reverse=True)
            
            c1=0
            for t in rankk:
              if c1<20:
                gg.append(t[0])
                c1=c1+1
            htt_rank[jz]=gg
            cn=cn+1
            if len(gg)>=5 and str(clp[0])==str(zero_shot_explainer.predicted_label):
              print("Hotel_Review_Id: "+str(jz)+"\n")
              #print(str())
              print("Predicted Sentiment Class: "+"\n"+str(clp[0])+"\n")
              print("Explanation Attention Words in the text to justify the prediction\n"+str(gg)+"\n")
              #  print("Visualize the Attention Words")
              # cls_explainer.visualize("distilbert_viz.html")
              print("\n\n")

              
htt_clsm={}
htt_rankm={}         
cnm=0
print("Movie Review")
for jz in mv:
  if cnm<200:#0000:
            #print("heloooooooooooooo")
            #print(ht[j])
            s=""
            zz=0
            htt_exp={}
            gg=[]
            for vv in mv[jz]:
              if len(vv)>=5:
                if zz<100:
                    if vv not in stop_words:
                        s=s+str(vv)+" "
                        zz=zz+1
            #print(j,str(s))
            #sys.exit()
            clp=review_explain(s)
            htt_clsm[jz]=clp[0]
            word_attributions = zero_shot_explainer(s,labels = [clp[0]],)

            #attp,expp=review_explain(s)
            #("Today apple released the new Macbook showing off a range of new features found in the proprietary silicon chip computer.")
            #if str(clp[0])==str(expp):
            #print(clp[0]) sorted(dc.items(), key=operator.itemgetter(1),reverse=True)
            #att_words= cls_explainer(s, class_name=clp[0].upper())
            #print("Predicted Sentiment Class: "+"\n"+str(clp[0])+"\n")
            from nltk.stem import WordNetLemmatizer
            lemmatizer = WordNetLemmatizer()
            from nltk.stem import PorterStemmer
            from nltk.tokenize import word_tokenize
            
            hhh=[]
            for hh in word_attributions[clp[0]]:#att_words:
              if hh[1]>0:
                if hh[0]. isalnum():
                  vvv=lemmatizer.lemmatize(str(hh[0]))
                  for bn in ht[jz]:
                    if bn.__contains__(str(vvv)):
                       if bn not in htt_exp:
                            htt_exp[bn]=float(hh[1])
            #print(htt_exp)
            #cn=cn+1
            #sys.exit()
            rankk=sorted(htt_exp.items(), key=operator.itemgetter(1),reverse=True)
            
            c1=0
            for t in rankk:
              if c1<20:
                gg.append(t[0])
                c1=c1+1
            htt_rankm[jz]=gg
            cnm=cnm+1
            if len(gg)>=5 and str(clp[0])==str(zero_shot_explainer.predicted_label):
              print("Movie_Review_Id: "+str(jz)+"\n")
              #print(str())
              print("Predicted Sentiment Class: "+"\n"+str(clp[0])+"\n")
              print("Explanation Attention Words in the text to justify the prediction\n"+str(gg)+"\n")
              #  print("Visualize the Attention Words")
              # cls_explainer.visualize("distilbert_viz.html")
              print("\n\n")
              
#print("\n\n")  


#return htt_cls,htt_rank,cls_explainer

#htt_cls,htt_rank,cls_explainer=hotel_review(ht)
'''
for tt in htt_cls:
  print("Hotel_Review_Id: "+str(tt)+"\n")
  #print(str())
  print("Predicted Sentiment Class: "+"\n"+str(htt_cls[tt])+"\n")
  print("Explanation Attention Words in the text to justify the prediction"+str(htt_rank[tt])+"\n")
  print("Visualize the Attention Words")
  cls_explainer.visualize("distilbert_viz.html")
  print("\n\n")

'''



Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Hotel Review
Hotel_Review_Id: 0

Predicted Sentiment Class: 
Negative

Explanation Attention Words in the text to justify the prediction
['recording', 'angry', 'vibrating', 'attach', 'tubes', 'nights', 'duplex', 'ceiling', 'peaceful', 'closed', 'stayed', 'constant', 'fridge', 'booking', 'booked', 'internal', 'place', 'placed', 'offered', 'specially']




Hotel_Review_Id: 1

Predicted Sentiment Class: 
Negative

Explanation Attention Words in the text to justify the prediction
['single', 'narrow', 'steps', 'story', 'difficult']




Hotel_Review_Id: 2

Predicted Sentiment Class: 
Negative

Explanation Attention Words in the text to justify the prediction
['manager', 'inconsistent', 'cleaned', 'clean', 'pretty', 'looked', 'pictures', 'midnight', 'white', 'shower', 'repairman'

'\nfor tt in htt_cls:\n  print("Hotel_Review_Id: "+str(tt)+"\n")\n  #print(str())\n  print("Predicted Sentiment Class: "+"\n"+str(htt_cls[tt])+"\n")\n  print("Explanation Attention Words in the text to justify the prediction"+str(htt_rank[tt])+"\n")\n  print("Visualize the Attention Words")\n  cls_explainer.visualize("distilbert_viz.html")\n  print("\n\n")\n\n'

In [3]:
!pip install transformers_interpret

     |████████████████████████████████| 1.4 MB 6.6 MB/s 
  Created wheel for transformers-interpret: filename=transformers_interpret-0.5.2-py3-none-any.whl size=23518 sha256=1fade466518fb08dddd4368265d37d03341871ca021e194ac5f9d4515bee25c6
  Stored in directory: /root/.cache/pip/wheels/18/8a/5f/db643e1cd44e7bb22c21b52b8fcaebadafaf0b361c33f05869
Successfully built transformers-interpret
